In [13]:
avIncomePerUser = 400 #сколько денег в среднем приносит один пользователь в месяц;
avSparePerUser = 100 #сколько денег в среднем вы будете вкладывать в удержание одного пользователя;
probAccept = 0.7 #с какой вероятностью пользователь примет ваше предложение;
topPercentToHold = 10 #сколько пользователей (например, топ 1% или топ 25% согласно ранжированию по вашей модели) будет участвовать в кампании.

In [ ]:
#будем рассчитывать доход от проведения удержания по каждому поьзователю (profitPerUser) так(он может быть отрицательным):
# incomePerUserNoAction = avIncomePerUser * (1-probChurn)
# incomePerUserAction = avIncomePerUser * probAccept - avSparePerUser
# profitPerUser = incomePerUserAction - incomePerUserNoAction;

In [1]:
#нужно найти вероятность ухода каждого пользователя, которого собираемся удерживать, обратимся для этого к модели
#cначала загрузим данные и подготовим модель с прошлой недели
import pandas as pd
import numpy as np
data = pd.read_csv('orange_small_churn_data.csv', delimiter =',')
data['labels'] = pd.read_csv('orange_small_churn_labels.csv', header=None)
#конвертируем колонку labels в int
data = data.astype({'labels': 'int32'})
#заменим все -1 в целевой переменной на 0
data['labels'] = data['labels'].map({1: 1, -1: 0})
from sklearn.model_selection import train_test_split
data, test = train_test_split(data, test_size=0.15, random_state=42)
target = np.array(data.iloc[:,-1])
target_test = np.array(test.iloc[:,-1])
#подготовим даные:
#выделим категориальные признаки (для baseline решения, возможно будет достаточно числовых)
numericalVarCount = 190
categorialVarCount = 40

data_num = data.iloc[:, 0:numericalVarCount]
#удалим числовые признаки, содержащие слишком большое количество NaN - значений
threshold = 0.7
NaN_frac = data_num.isna().sum(axis = 0)/data_num.shape[0]
numVarsToStay = list(NaN_frac[NaN_frac < threshold].index)
data_num = data_num.loc[:,numVarsToStay]
#Перед построением моделей, подготовим данные: заменим NaN на медианные значения,
medians = data_num.median()
data_num.fillna(medians, inplace=True)
#выполним стандартизацию числовых признаков
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_num = scaler.fit_transform(data_num)

# #обработаем категориальные признаки методикой one-hot-encoding
from sklearn.feature_extraction import DictVectorizer as DV
data_cat = data.iloc[:, numericalVarCount:-1]

data_cat_oh = pd.get_dummies(data_cat, dummy_na=True, drop_first=True)
NaN_frac = data_cat.isna().sum(axis = 0)/data_cat.shape[0]
NaN_frac
threshold = 0.1
NaN_frac = data_cat.isna().sum(axis = 0)/data_num.shape[0]
catVarsToStay = list(NaN_frac[NaN_frac < threshold].index)
data_cat = data_cat.loc[:,catVarsToStay]
data_cat = data_cat.fillna('NA').astype(str)

#Подсчитаем количество уникальных значений в категориальных признаках, от этого будет зависеть способ кодировки
unique_counts = []
for c in data_cat.columns:
    unique_counts.append(data_cat[c].dropna().unique().shape[0])
cat_unique = pd.DataFrame()
cat_unique['unique_counts'] = unique_counts
cat_unique.index = data_cat.columns
cat_unique.sort_values(by='unique_counts', ascending=False)
cat_unique

cat_feat_for_OHE = list(cat_unique[cat_unique['unique_counts'] < 50].index)
cat_feat_for_OHE
encoder = DV(sparse = False)
data_cat_oh = encoder.fit_transform(data_cat[cat_feat_for_OHE].T.to_dict().values())

data_all = np.hstack((data_num,data_cat_oh))
data_all.shape

#выполняем на ней ту же обработку, что для набора обучения
test_num = test.iloc[:, 0:numericalVarCount]
#выкинем признаки, которые выкидывали при обучении
test_num = test.loc[:,numVarsToStay]
#заполним NaN медианными значениями train!!! набора
test_num.fillna(medians, inplace=True)
#выполним стандартизацию, с теми же параметрами, что при обучении:
test_num = scaler.transform(test_num)
test_cat = test.iloc[:, numericalVarCount:-1]
test_cat = test_cat.loc[:,catVarsToStay]
test_cat = test_cat.fillna('NA').astype(str)
test_cat_oh = encoder.transform(test_cat[cat_feat_for_OHE].T.to_dict().values())
test_all = np.hstack((test_num,test_cat_oh))


#будем использовать следующие модели для baseline решения:
#RandomForestClassifier, RidgeClassifier и SGDClassifier
#ввиду несбалансированности выборок везде используем class_weight='balanced'
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
clf = SGDClassifier(loss = 'log', class_weight='balanced', max_iter=1000, tol=1e-4, alpha=0.01, random_state=42)
clf.fit(data_all, target)
scores = cross_val_score(clf, data_all, target, cv=3, scoring = 'f1')
basic_score = scores.mean()
basic_score

0.20253190381202768

In [6]:
#для всех оценок будем использовать тестовую выборку
probs = clf.predict_proba(test_all)[:,1]
probs

array([ 0.35513105,  0.19989424,  0.41336346, ...,  0.74328987,
        0.33162091,  0.5486033 ])

In [12]:
#соритируем вероятности по убыванию
probs[::-1].sort()
#и берем определенный процент пользователей, которых собираемся удерживать
probs = probs[:round(probs.shape[0] * topPercentToHold/100.0)]
probs.shape

(600,)

In [14]:
#посчитаем с заданными параметрами доход от проведения кампании по удержанию:
probChurn = probs
incomePerUserNoAction = avIncomePerUser * (1-probChurn)
incomePerUserAction = avIncomePerUser * probAccept - avSparePerUser
profitPerUser = incomePerUserAction - incomePerUserNoAction;

Задание 1: Итоговая прибыль по проведению кампании по удержанию, если использовать параметры из начала ноутбука:

In [16]:
np.sum(profitPerUser)

48949.661171431086

Задание 2: Для ответа на вопрос, посмотрим, начиная с какого процента пользователей, начнутся убытки:

In [22]:
#применим те же рассчеты, что и ранее, только изменим процент пользователей
topPercentToHold = 100
probs = clf.predict_proba(test_all)[:,1]
probs[::-1].sort()
probs = probs[:round(probs.shape[0] * topPercentToHold/100.0)]
probChurn = probs
incomePerUserNoAction = avIncomePerUser * (1-probChurn)
incomePerUserAction = avIncomePerUser * probAccept - avSparePerUser
profitPerUser = incomePerUserAction - incomePerUserNoAction;
percent = np.sum(profitPerUser > 0)/profitPerUser.shape[0]
percent

0.36733333333333335

Таким образом, экономически выгодно проводить компанию по топ-37 процентам пользователей

Задание 3: Добавим еще параметры: непредвиденные расходы, вероятность их возникновения, непредвиденные доходы и их вероятность
также изменим стоимость удержания и вероятность принятия пользователем предложения

In [23]:
avSparePerUser = 300 #сколько денег в среднем вы будете вкладывать в удержание одного пользователя;
probAccept = 0.5 #с какой вероятностью пользователь примет ваше предложение;
UnexpectedIncomeProb = 0.6 #вероятность непредвиденной прибыли на пользователя
UnexpectedSpareProb = 0.5 #вероятность непредвиденных расходов на пользователя
UnexpectedIncome = 50 #размер непредвиденной прибыли на пользователя
UnexpectedSpare = 60 #размер непредвиденных расходов на пользователя

WeightedUnexpectedIncome = UnexpectedIncome * UnexpectedIncomeProb
WeightedUnexpectedSpare = UnexpectedSpare * UnexpectedSpareProb

topPercentToHold = 100
probs = clf.predict_proba(test_all)[:,1]
probs[::-1].sort()
probs = probs[:round(probs.shape[0] * topPercentToHold/100.0)]
probChurn = probs
incomePerUserNoAction = avIncomePerUser * (1-probChurn)
incomePerUserAction = avIncomePerUser * probAccept - avSparePerUser + WeightedUnexpectedIncome - WeightedUnexpectedSpare
profitPerUser = incomePerUserAction - incomePerUserNoAction;
percent = np.sum(profitPerUser > 0)/profitPerUser.shape[0]
percent

0.0

Видим, что из-за увеличения стоимости удержания и уменьшения вероятности принятия предложения, кампанию стало проводить невыгодно: процент пользователей, с которых можно получить пользу от проведения компании теперь равен 0

Задание 4: Применение модели выгодно не всегда, например, параметры, при которых невыгодно проводить кампанию:

avSparePerUser = 300 #сколько денег в среднем вы будете вкладывать в удержание одного пользователя;

probAccept = 0.5 #с какой вероятностью пользователь примет ваше предложение;

UnexpectedIncomeProb = 0.6 #вероятность непредвиденной прибыли на пользователя

UnexpectedSpareProb = 0.5 #вероятность непредвиденных расходов на пользователя

UnexpectedIncome = 50 #размер непредвиденной прибыли на пользователя

UnexpectedSpare = 60 #размер непредвиденных расходов на пользователя


Задание 5: После проведения тестирования, выяснилось, что при улучшении качества модели (по метрике f1_score) и фиксированных
остальных параметрах на 1% и 3% можно добиться увеличения прибыли на 0.5835% и 0.9436% соответственно по сравнению с прибылью из пункта 1

Задание 6: Ответ на вопрос зависит от параметров экономической модели. Если использовать параметры, как в задании 1, то вложение
средств является экономически оправданным, но, как видно из результатов в задании 4, чтобы существенно повысить прибыль, при использовании модели, нужно очень сильно повысить качество предсказываемых вероятностей оттока, что не представляется возможным.